# Dropbox Authentication and File Upload

This notebook demonstrates how to authenticate with Dropbox using refresh tokens and upload files programmatically.

## Step 1: Load Environment Variables

Load your Dropbox credentials from the `.env` file. Make sure you have `APP_KEY`, `APP_SECRET`, and `REFRESH_TOKEN` defined in your environment.

In [1]:
import dotenv

dotenv.load_dotenv()

True

## Step 2: Define the Upload Function

This function handles the authentication to Dropbox using OAuth2 refresh tokens and uploads a file to your Dropbox account. It requires app credentials and a refresh token for authentication.

In [ ]:
import os
from pathlib import Path

import dropbox
from dropbox.exceptions import ApiError


def upload_to_dropbox(
    local_file_path: str | Path,
    dropbox_path: str,
    overwrite: bool = True,
) -> str:
    """
    Upload a file to Dropbox using refresh token authentication.

    Args:
        local_file_path: Path to the local file to upload
        dropbox_path: Destination path in Dropbox (e.g., '/photos/image.jpg')
        overwrite: If True, overwrite existing file at destination

    Returns:
        The Dropbox file path of the uploaded file

    Raises:
        ValueError: If required environment variables are missing
        FileNotFoundError: If local file doesn't exist
        ApiError: If Dropbox API call fails
    """
    # Get credentials from environment variables
    app_key = os.getenv("APP_KEY")
    app_secret = os.getenv("APP_SECRET")
    refresh_token = os.getenv("REFRESH_TOKEN")

    if not all([app_key, app_secret, refresh_token]):
        raise ValueError(
            "Missing required environment variables: "
            "APP_KEY, APP_SECRET, or REFRESH_TOKEN"
        )

    # Convert to Path object and verify file exists
    local_file_path = Path(local_file_path)
    if not local_file_path.exists():
        raise FileNotFoundError(f"Local file not found: {local_file_path}")

    # Create Dropbox client with refresh token
    dbx = dropbox.Dropbox(
        app_key=app_key,
        app_secret=app_secret,
        oauth2_refresh_token=refresh_token,
    )

    # Read file content
    with open(local_file_path, "rb") as f:
        file_content = f.read()

    # Upload to Dropbox
    try:
        result = dbx.files_upload(
            f=file_content,
            path=dropbox_path,
            mode=dropbox.files.WriteMode("overwrite" if overwrite else "add"),
            autorename=False,
        )
        return result.path_display
    except ApiError as e:
        raise ApiError(e.request_id, e.error, e.user_message) from e

## Step 3. Get a Refresh Token with Correct Scopes

Run this cell to generate an authorization URL. Visit the URL, authorize the app, and you'll get a code to exchange for a refresh token with `files.content.write` scope.

In [ ]:
import urllib.parse

APP_KEY = os.getenv("APP_KEY")
APP_SECRET = os.getenv("APP_SECRET")

# Scopes needed: files.content.write (and optionally files.content.read)
SCOPES = ["files.content.write", "files.content.read"]

# Generate authorization URL
auth_url = (
    f"https://www.dropbox.com/oauth2/authorize?"
    f"client_id={APP_KEY}&"
    f"response_type=code&"
    f"token_access_type=offline&"
    f"scope={urllib.parse.quote(' '.join(SCOPES))}"
)

print("Authorization URL:")
print(auth_url)
print("\n1. Visit the URL above in your browser")
print("2. Click 'Allow'")
print("3. Copy the authorization code")
print("4. Paste it in the next cell and run it")

In [ ]:
import requests

# Replace with the authorization code from the URL redirect
AUTH_CODE = "YOUR_AUTH_CODE_HERE"

# Exchange authorization code for refresh token
token_url = "https://api.dropboxapi.com/oauth2/token"
data = {
    "code": AUTH_CODE,
    "grant_type": "authorization_code",
    "client_id": APP_KEY,
    "client_secret": APP_SECRET,
}

response = requests.post(token_url, data=data)
token_data = response.json()

if "refresh_token" in token_data:
    refresh_token = token_data["refresh_token"]
    print(f"✓ New Refresh Token (with files.content.write scope):")
    print(f"REFRESH_TOKEN = {refresh_token}")
    print(f"\nUpdate your .env file with this token")
else:
    print("Error:", token_data.get("error", "Unknown error"))
    print(token_data)

## Step 4: Test the Upload Function

Test uploading a file to Dropbox using your credentials. Replace the file path with a file you want to upload.

In [3]:
upload_to_dropbox(
    local_file_path="/Users/chris/Downloads/Screenshot 2026-01-28 at 16.21.59.png",
    dropbox_path="/Apps/pyclops/screenshot.jpg",
    overwrite=True,
)

'/Apps/pyclops/screenshot.jpg'